# Numerical Data - Cleanup

<br>
**Author** Mary Letey <br>
**Date** 4/16/18 <br>
**File** megaset.ipynb <br>
**Purpose** Creating a mega-dataset aggregating all previous data sets, sorted by date for the last 5 years

<br><br>

In [35]:
# Imports
import numpy as np
import pandas as pd
import datetime
from dateutil import parser
import matplotlib.pylab as plt
%matplotlib inline

In [115]:
# -- dell seeking alpha
df = pd.read_csv("../data/scraper/dell_5yrs_seekingalpha.csv");
dates = list(df["date"]);

newdelldates = []
for i in range(0,len(dates)):
    mystr = dates[i]
    newdelldates.append(parser.parse(mystr))
    
delldates1 = newdelldates;


dftotal = df;
dftotal["date"] = delldates1;


In [111]:
df.head()

,title,date,body
0,"Dude, You're Not Getting DELL...Or AUTO",2013-03-18 00:00:00,(At Least Not At These Prices)Despite differen...
1,Critical Analysis Of Dell Buyout Plan Shows Hi...,2013-03-14 00:00:00,The media is currently focused on the saga of ...
2,The Value Of Dell Using The EBIT Multiple Valu...,2013-03-14 00:00:00,Let me go through a EBIT multiple valuation me...
3,Dell LBO Deal Structure - Like Buying A Rental...,2018-02-07 00:00:00,Henry Blodget is pulling on exactly the right ...
4,Billionaire Carl Icahn Moving Into Multilevel-...,2013-03-12 08:55:00,"In the view of many investors, billionaire Car..."


In [91]:
a = [1,2,3,4,5,6,7,8,9]

visited = 0
index = 0
l = len(a)
print("Total = ",l)
while visited < l-(3-1):
    print(a)
    print("i=",index)
    if a[index]%3 == 0:
        del a[index]
#         index -= 1
    index += 1
    visited += 1
    print("we have vistited",visited,"elements")

print(a)
    
    


Total =  9
[1, 2, 3, 4, 5, 6, 7, 8, 9]
i= 0
we have vistited 1 elements
[1, 2, 3, 4, 5, 6, 7, 8, 9]
i= 1
we have vistited 2 elements
[1, 2, 3, 4, 5, 6, 7, 8, 9]
i= 2
we have vistited 3 elements
[1, 2, 4, 5, 6, 7, 8, 9]
i= 3
we have vistited 4 elements
[1, 2, 4, 5, 6, 7, 8, 9]
i= 4
we have vistited 5 elements
[1, 2, 4, 5, 7, 8, 9]
i= 5
we have vistited 6 elements
[1, 2, 4, 5, 7, 8, 9]
i= 6
we have vistited 7 elements
[1, 2, 4, 5, 7, 8]
